In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [28]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [29]:
cat = Table.read('./Catalog/NSA/z002_004/nsa_cutout_cat_z002_004.fits')

### Generate RGB images

In [31]:
from kuaizi.display import display_HSC_cutout_rgb

In [32]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/'
WEBPAGE_FIG_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/figure/'

In [34]:
problem_ids = []
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            #print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")
            problem_ids.append(obj['viz-id'])
            
if not os.path.isdir(FIGURE_DIR):
    os.mkdir(FIGURE_DIR)

In [35]:
print('# of no cutouts:', np.unique(problem_ids).shape)

# of no cutouts: (71,)


In [36]:
# Display each cutout
for i, obj in enumerate(cat[1100:1200]):
    ind = obj['viz-id']
    try:
        images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'griz'])
        fig, ax = plt.subplots(figsize=(5, 5))
        #size = obj['rhalf_ellip'] * 5 if obj['rhalf_ellip'] * 5 > 100 else 100
        ax = display_HSC_cutout_rgb(images, ax=ax, half_width=None)
        plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(ind, e)
        
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 5476
Progress: 10 / 5476
Progress: 20 / 5476
Progress: 30 / 5476
Progress: 40 / 5476
Progress: 50 / 5476
Progress: 60 / 5476
Progress: 70 / 5476
Progress: 80 / 5476
Progress: 90 / 5476


In [37]:
# Copy figures in FIGURE_DIR to WEBPAGE_FIG_DIR
if not os.path.isdir(WEBPAGE_FIG_DIR):
    os.mkdir(WEBPAGE_FIG_DIR)
else:
    import glob
    fileList = glob.glob(WEBPAGE_FIG_DIR + '*.png', recursive=True)
    [os.remove(file) for file in fileList];
            
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try: 
        copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_FIG_DIR + f'nsa_{ind}_cutout.png')
    except Exception as e:
        print(ind, e)

1655 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_1655_cutout.png'
1660 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_1660_cutout.png'
1667 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_1667_cutout.png'
1670 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_1670_cutout.png'
1699 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_1699_cutout.png'
1719 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_1719_cutout.png'
1720 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_1720_cutout.png'
1733 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_1733_cutout.png'
1747 [Errno 2] N

20316 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_20316_cutout.png'
20319 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_20319_cutout.png'
20328 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_20328_cutout.png'
20340 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_20340_cutout.png'
20367 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_20367_cutout.png'
20375 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_20375_cutout.png'
20425 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_20425_cutout.png'
20426 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_20426_cutout.png'


42704 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_42704_cutout.png'
42720 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_42720_cutout.png'
42728 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_42728_cutout.png'
42735 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_42735_cutout.png'
42738 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_42738_cutout.png'
42776 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_42776_cutout.png'
42796 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_42796_cutout.png'
42813 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_42813_cutout.png'


63454 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_63454_cutout.png'
63484 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_63484_cutout.png'
63495 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_63495_cutout.png'
63520 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_63520_cutout.png'
63573 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_63573_cutout.png'
63581 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_63581_cutout.png'
63594 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_63594_cutout.png'
63600 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_63600_cutout.png'


83617 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_83617_cutout.png'
83639 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_83639_cutout.png'
83649 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_83649_cutout.png'
83669 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_83669_cutout.png'
83681 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_83681_cutout.png'
83742 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_83742_cutout.png'
83764 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_83764_cutout.png'
83779 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_83779_cutout.png'


### Generate webpage for cutouts. We use `diezi/webpage/gen_web_cutout.py`

In [38]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [39]:
#FOR NSA SAMPLE
WEBPAGE_FIG_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/figure/'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/page/'
SCARLET_DIR = None #'/tigress/jiaxuanl/public_html/NSA/scarlet_result/'
sample_name = 'NSA'

In [40]:
webpage_cutout(WEBPAGE_FIG_DIR, WEBPAGE_DIR, sample_name, SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 100 galaxies to be displayed
Total pages: 5


In [41]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/page/


### The search box on webpage still doesn't work. Need to work on the javascript file